In [2]:
import transformers
import tensorflow as tf

In [3]:
#load the model monologg/bert-base-cased-goemotions-original from huggingface
model = transformers.AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
tokenizer = transformers.AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model.cuda()

c:\Users\mlar5\anaconda3\envs\CudaTest\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mlar5\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [5]:
#test the model on a sentence
sentence = "I am so happy"
tokens = tokenizer.encode(sentence, return_tensors="pt")
tokens = tokens.cuda()
result = model(tokens)

In [13]:
#identify the top 3 emotion with the highest probability
emotion = result.logits
emotion = emotion.cpu().detach().numpy()
emotion = emotion[0]
emotion = emotion.argsort()[-3:][::-1]
emotion = emotion.tolist()
print(emotion)

[17, 0, 13]


In [24]:
#convert model.config.id2label to a list of emotions, where the key is the index of the emotion
emotion_dict = model.config.id2label

In [26]:
#identify the label of top 3 emotions from emotion list
for i in emotion:
    print(emotion_dict[i])


joy
admiration
excitement


In [100]:
def getTopNEmotions(sentence, n,printScores=False):
    tokens = tokenizer.encode(sentence, return_tensors="pt")
    tokens = tokens.cuda()
    result = model(tokens)
    if printScores:
        #print(result.logits)
        #apply the softmax function to the logits
        #convert the results to cpu and detach them from the graph
        #convert the results to numpy array
        #apply the argsort function to the array
        #get the top n emotions
        #convert the results to list
        emotion = result.logits
        #print(emotion[0])

        emotion = emotion.cpu().detach().numpy()
        #print(emotion[0])
        softmax = tf.nn.softmax(emotion[0])
    else:
        emotion = result.logits
        emotion = emotion.cpu().detach().numpy()
        emotion = emotion[0]
        softmax = tf.nn.softmax(emotion)
        emotion = emotion.argsort()[-n:][::-1]
        emotion = emotion.tolist()

        emotion_dict = model.config.id2label
        id=0
        for i in emotion:
            print(emotion_dict[i])
            print(softmax[emotion[id]].numpy()*100,"%")
            id+=1
    return

In [8]:
#identify the emotion with the highest probability
emotion = result.logits.argmax()
print(emotion)
#which emotion is that?
print(model.config.id2label[emotion.item()])

tensor(17, device='cuda:0')
joy


In [28]:
emotion_dict.values()
emotionGroups=[]

dict_values(['admiration', 'amusement', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'anger', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment'])

In [30]:
emotionSet = set(emotion_dict.values())

In [31]:
approvedMoods=['neutral']

In [109]:
unclear2 = "'Cause you see people, people, people, people Don't really know you (They don't really know you) They don't really know you 'Cause you see people, people, people They don't really know you (Mmm) They don't really know—"
unclear = "I've been drinking more alcohol for the past five days Did you check on me? Now, did you look for me? I walked in the room, eyes are red and I don't smoke banga Did you check on me? (Did you check on me?) Now, did you notice me?"

In [54]:
lyrics = "She just hit my phone, she said, 'Tecca, you a winner' (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I’m a star I just wanna ball with my guys Drive off the Wock', he just caught a DUI He think he know the answers, nigga, like he Bill Nye"

In [107]:
sadLyrics = "Hello darkness, my old friend, I've come to talk with you again, because a vision softly creeping, left its seeds while I was sleeping."

In [110]:
getTopNEmotions(unclear2, 3)

neutral
99.9980092048645 %
annoyance
0.00058388282013766 %
disappointment
0.0003420888106120401 %
